[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"

sentiment = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='latin-1')

In [3]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [4]:
sentiment.sample(10)

,id,sentiment,review
24080,57_10,1,I have to say that this is one of the best mov...
10362,4355_10,1,I also viewed this film at the Santa Barbara F...
5409,6000_1,0,"Johnathan Frakes is a good actor and, when he'..."
11783,8164_1,0,I saw this movie at an actual movie theater (p...
13473,11640_4,0,"Pier Paolo Pasolini, or Pee-pee-pee as I prefe..."
17918,9041_8,1,"Based on fact, this is the story of a teenager..."
12110,10639_7,1,Dolph Lundgren is back! Detention marks Dolphs...
9128,10259_1,0,I saw this movie in the theater when I was a k...
2352,3690_1,0,Baba - Rajinikanth will never forget this name...
11913,513_1,0,Burt Reynolds came to a point in his career wh...


## 2. Preprocessing

In [5]:
# stop words

import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chikara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Removing special characters and "trash"
import re

def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))

    return text

 
str = 'aaaaAAAaa    bbbbb <br> ccc :) ccc'

print(str)
print(preprocessor(str))

aaaaAAAaa    bbbbb <br> ccc :) ccc
aaaaaaaaa bbbbb ccc ccc :)


In [7]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer

porter = PorterStemmer()


def tokenizer_porter(text):
    token = text.split(' ')
    
    new = []
    for word in token:
        new.append(porter.stem(word))
    
    return new

print('Hi there, I am loving this, like with a lot of love')
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

Hi there, I am loving this, like with a lot of love
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [8]:
sentiment.head(10)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
5,8196_8,1,I dont know why people think this is such a ba...
6,7166_2,0,"This movie could have been very good, but come..."
7,10633_1,0,I watched this video at a friend's house. I'm ...
8,319_1,0,"A friend of mine bought this film for Â£1, and..."
9,8713_10,1,<br /><br />This movie is full of references. ...


In [14]:
# split the dataset in train and test
# Your code here

from sklearn.model_selection import train_test_split

X = sentiment['review']
y = sentiment['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [15]:
# Import Pipeline, LogisticRegression, TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stop = stopwords.words('english')

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## 4. Evaluate Model

In [11]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)

acc = accuracy_score(predictions, y_test)

0.8857333333333334


In [19]:
print(acc)

0.8857333333333334


In [18]:
cfn = confusion_matrix(predictions, y_test)
print(cfn)

[[3297  358]
 [ 499 3346]]


In [20]:
report = classification_report(predictions, y_test)
print(report)

             precision    recall  f1-score   support

          0       0.87      0.90      0.88      3655
          1       0.90      0.87      0.89      3845

avg / total       0.89      0.89      0.89      7500



## 5. Export Model 

In [26]:
# Using pickle to export our trained model
import pickle
import os



In [28]:
pickle.dump(clf, open(os.path.join('dump.pkl'), 'wb'), protocol=4)
